In [1]:
import pandas as pd
import numpy as np
import scipy
import sklearn

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel, SelectPercentile, chi2, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import metrics

In [4]:
import optuna
from optuna.visualization import plot_intermediate_values

In [5]:
import numpy as np
import xgboost as xgb
#import lightgbm as lgb

In [6]:
SEED = 85

In [7]:
from hypster_xgboost import *
from hypster import *

# Get Dataset

In [8]:
from scipy.sparse import csr_matrix, save_npz, load_npz

In [9]:
dataset = "boston" #adult, boston

In [10]:
if dataset=="adult":
    X_train = pd.read_pickle("./data/adult_X_train.pkl")
    y_train = pd.read_pickle("./data/adult_y_train.pkl")
    X_test = pd.read_pickle("./data/adult_X_test.pkl")
    y_test = pd.read_pickle("./data/adult_y_test.pkl")
    cat_columns = X_train.select_dtypes(include="object").columns
elif dataset=="newsgroup":
    X_train = load_npz("./data/X_train.npz")
    y_train = pd.read_pickle("./data/y_train.pkl")
    X_test = load_npz("./data/X_test.npz")
    y_test = pd.read_pickle("./data/y_test.pkl")
    cat_columns=None
else:
    X_train = pd.read_pickle("./data/boston_X_train.pkl")
    y_train = pd.read_pickle("./data/boston_y_train.pkl")
    X_test = pd.read_pickle("./data/boston_X_test.pkl")
    y_test = pd.read_pickle("./data/boston_y_test.pkl")
    cat_columns = None

In [11]:
#X_train = X_train.sample(n=10000, random_state=SEED, axis=0)

In [12]:
#y_train = y_train.iloc[X_train.index].reset_index(drop=True)
#X_train.reset_index(inplace=True, drop=True)

In [13]:
#pipeline - pipeline_objective OR regular pipeline
#consider making pre-made steps with best practices (FS, scaling, etc...) then add option to concat to make one pipeline 

In [14]:
#pipeline = Pipeline([("sel", SelectPercentile(chi2))])
#pipe_params = {"sel__percentile" : optuna.distributions.IntUniformDistribution(1,100)}
pipeline = None
pipe_params = None

In [15]:
from optuna.samplers import TPESampler

In [16]:
sampler = TPESampler(**TPESampler.hyperopt_parameters())

In [17]:
#sampler = optuna.integration.CmaEsSampler()

# Regressor

In [18]:
#TODO: automatic seed
xgb_linear = XGBRegressorHypster(booster_list=['gblinear']
                               ,param_dict={'nthread' : 1
                                            #TODO check what happens when you run over parameters in optuna
                               #,'subsample' : 0.9
                               }
                               )
xgb_tree = XGBRegressorHypster(booster_list=['gbtree', 'dart'], 
                               param_dict={'max_depth' : optuna.distributions.IntUniformDistribution(2, 20)
                                               ,'nthread' : 1
                                               #,'subsample' : 0.9 
                                               })
#lgb_estimator = LGBClassifierOptuna()
#sgd_estimator = SGDClassifierOptuna()
#rf_estimator  = RFClassifierOptuna()

In [19]:
estimators = [xgb_linear, xgb_tree]#, sgd|_estimator]

In [20]:
reg = HyPSTERRegressor(estimators, pipeline, pipe_params,
                        scoring="neg_mean_squared_error", cv=KFold(n_splits=3, random_state=SEED), sampler=sampler, 
                        refit=False, random_state=SEED, n_jobs=-1)

In [21]:
%%time
reg.fit(X_train, y_train, cat_columns=cat_columns, n_trials_per_estimator=[10, 60])

XGBoost Regressor
Score: 568.705
Score: 84.99023
Score: 216.46002
XGBoost Regressor
Score: 296.68948
Score: 462.12625


[W 2019-08-06 15:00:43,151] Setting status of trial#4 as TrialState.FAIL because of the following error: ValueError("Input contains NaN, infinity or a value too large for dtype('float32').")
Traceback (most recent call last):
  File "/Users/giladrubin/anaconda3/lib/python3.7/site-packages/optuna/study.py", line 469, in _run_trial
    result = func(trial)
  File "/Users/giladrubin/python_workspace/HyPSTER/hypster.py", line 136, in __call__
    fold_score = estimator.score_test(self.scoring, self.scorer_type)
  File "/Users/giladrubin/python_workspace/HyPSTER/hypster_xgboost.py", line 256, in score_test
    return scorer(self.dtest.get_label(), preds)
  File "/Users/giladrubin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/regression.py", line 239, in mean_squared_error
    y_true, y_pred, multioutput)
  File "/Users/giladrubin/anaconda3/lib/python3.7/site-packages/sklearn/metrics/regression.py", line 77, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False)
  Fi

Score: 432.04593
Score: 117.13271
Score: 453.34067
Score: 495.48026
Score: 386.08017
Score: 424.19736
Score: 185.61485
Score: 186.16032
Score: 178.03967
Score: 204.28264
Score: 118.57308
Score: 33.77169
Score: 54.69024
Score: 379.35376
Score: 373.43057
Score: 403.01633
Score: 200.26775
Score: 51.98082
Score: 57.20684
Score: 443.6737
CPU times: user 50.3 s, sys: 4.58 s, total: 54.9 s
Wall time: 51.3 s


In [22]:
reg.best_score_

33.77168655395508

In [23]:
reg.best_params_

{'max_depth': 9,
 'init_eta': 0.890493938280948,
 'booster': 'gbtree',
 'lambda': 7.011497445818966e-08,
 'alpha': 1.0124912472584068e-08,
 'min_child_weight': 1,
 'gamma': 2.938959433727388e-09,
 'grow_policy': 'depthwise',
 'subsample': 0.50715024824889,
 'colsample_bytree': 0.3654668161167628,
 'colsample_bynode': 0.10903004655336655,
 'forest_boosting': True,
 'num_parallel_tree': 11}

In [24]:
reg.refit(X_train, y_train)

In [25]:
test_preds = reg.predict(X_test)

In [26]:
sklearn.metrics.mean_absolute_error(y_test, test_preds)

3.813087014523809

In [27]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test, test_preds))

5.8280181821734685

# Classifier

In [28]:
dataset = "newsgroup" #adult, boston

In [29]:
if dataset=="adult":
    X_train = pd.read_pickle("./data/adult_X_train.pkl")
    y_train = pd.read_pickle("./data/adult_y_train.pkl")
    X_test = pd.read_pickle("./data/adult_X_test.pkl")
    y_test = pd.read_pickle("./data/adult_y_test.pkl")
    cat_columns = X_train.select_dtypes(include="object").columns
elif dataset=="newsgroup":
    X_train = load_npz("./data/X_train.npz")
    y_train = pd.read_pickle("./data/y_train.pkl")
    X_test = load_npz("./data/X_test.npz")
    y_test = pd.read_pickle("./data/y_test.pkl")
    cat_columns=None
else:
    X_train = pd.read_pickle("./data/boston_X_train.pkl")
    y_train = pd.read_pickle("./data/boston_y_train.pkl")
    X_test = pd.read_pickle("./data/boston_X_test.pkl")
    y_test = pd.read_pickle("./data/boston_y_test.pkl")
    cat_columns = None

In [30]:
#TODO: automatic seed
xgb_linear = XGBClassifierHypster(booster_list=['gblinear']
                               ,param_dict={'nthread' : 1
                                            #TODO check what happens when you run over parameters in optuna
                               #,'subsample' : 0.9
                               }
                               )
#gb_dart = XGBClassifierHypster(booster_list=['dart'])
#xgb_tree = XGBClassifierHypster(booster_list=['gbtree', 'dart'], user_param_dict={'max_depth' : 2})
xgb_tree = XGBClassifierHypster(booster_list=['gbtree', 'dart'], 
                               param_dict={'max_depth' : optuna.distributions.IntUniformDistribution(2, 20)
                                               ,'nthread' : 1
                                               #,'subsample' : 0.9 
                                               })
#lgb_estimator = LGBClassifierOptuna()
#sgd_estimator = SGDClassifierOptuna()
#rf_estimator  = RFClassifierOptuna()

In [31]:
estimators = [xgb_linear, xgb_tree]#, sgd|_estimator]

In [35]:
clf = HyPSTERClassifier(estimators, pipeline, pipe_params,
                        scoring="roc_auc", cv=StratifiedKFold(n_splits=3, random_state=SEED), #sampler=sampler, 
                        refit=False, random_state=SEED, n_jobs=-1)

In [36]:
%%time
clf.fit(X_train, y_train, cat_columns=cat_columns, n_trials_per_estimator=10)

XGBoost Classifier
Score: 0.94655
Score: 0.95792
Score: 0.96429
Score: 0.21535
Score: 0.92812
XGBoost Classifier
Score: 0.77281
Score: 0.84839
Score: 0.90147
CPU times: user 12min 56s, sys: 7.38 s, total: 13min 3s
Wall time: 4min 42s


In [37]:
clf.best_score_

0.9642894704537751

In [38]:
clf.best_params_

{'scale_pos_weight': 1.0,
 'init_eta': 0.3289169512379114,
 'booster': 'gblinear',
 'lambda': 1.853738975157325e-07,
 'alpha': 1.831995364835684e-09,
 'shotgun_feature_selector': 'cyclic'}

In [39]:
clf.refit(X_train, y_train)

In [40]:
test_preds = clf.predict(X_test)

In [41]:
sklearn.metrics.accuracy_score(y_test, test_preds)

0.9332414336352894

In [42]:
test_probs = clf.predict_proba(X_test)
test_probs = test_probs[:,1]

In [43]:
sklearn.metrics.roc_auc_score(y_test, test_probs)

0.9669730108734756

# Misc